# CityBikes

Send a request to CityBikes for the city of your choice.

In [7]:
# imports

import requests
import json # in vscode, this import is needed to parse the JSON from the api call
import pandas as pd
import numpy as np

# from IPython.display import JSON # for nice JSON (or dict) display in JupyterLab (not currently supported in Jupyter Notebook)

Contact the server via an API call

API Details (Added by my for reference): https://api.citybik.es/v2/

In [9]:
# set city for investigation - Toronto AKA Bike Share Toronto
city_name = "Toronto, ON"

# get network response for all bike share networks in the world first
response = requests.request('GET', url="https://api.citybik.es/v2/networks")
data_networks = response.json()

# normalize it into a dataframe
df_networks_raw = pd.json_normalize(data_networks['networks'])
df_networks_raw

,company,href,id,name,location.city,location.country,location.latitude,location.longitude,source,gbfs_href,ebikes,license.name,license.url
0,[ЗАО «СитиБайк»],/v2/networks/velobike-moscow,velobike-moscow,Velobike,Moscow,RU,55.750000,37.616667,NaN,NaN,NaN,NaN,NaN
1,[Comunicare S.r.l.],/v2/networks/bicincitta-siena,bicincitta-siena,Bicincittà,Siena,IT,43.318600,11.330600,https://www.bicincitta.com/frmLeStazioni.aspx?...,NaN,NaN,NaN,NaN
2,[Cyclopolis Systems],/v2/networks/cyclopolis-maroussi,cyclopolis-maroussi,Cyclopolis,Maroussi,GR,38.056872,23.808330,NaN,NaN,NaN,NaN,NaN
3,[Cyclopolis Systems],/v2/networks/cyclopolis-nafplio,cyclopolis-nafplio,Cyclopolis,Nafplio,GR,37.563940,22.809340,NaN,NaN,NaN,NaN,NaN
4,[Comunicare S.r.l.],/v2/networks/bicincitta-parco-dei-colli-di-ber...,bicincitta-parco-dei-colli-di-bergamo,Bicincittà,Parco dei Colli di Bergamo,IT,45.722956,9.649230,https://www.bicincitta.com/frmLeStazioni.aspx?...,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
785,[PickeBike],/v2/networks/pickebike-aubonne,pickebike-aubonne,PickeBike Aubonne,Aubonne,CH,46.494900,6.396300,https://github.com/SFOE/sharedmobility/blob/ma...,https://gbfs.prod.sharedmobility.ch/v2/gbfs/pi...,NaN,Open Use,https://opendata.swiss/en/dataset/standorte-un...
786,"[Mobilitätsakademie AG, Touring Club Switzerla...",/v2/networks/carvelo,carvelo,carvelo eCargo-Bike Sharing,Switzerland,CH,46.982900,8.233100,https://github.com/SFOE/sharedmobility/blob/ma...,https://gbfs.prod.sharedmobility.ch/v2/gbfs/ca...,NaN,Open Use,https://opendata.swiss/en/dataset/standorte-un...
787,[PickeBike],/v2/networks/pickebike-basel,pickebike-basel,PickeBike Basel,Basel,CH,47.557900,7.590200,https://github.com/SFOE/sharedmobility/blob/ma...,https://gbfs.prod.sharedmobility.ch/v2/gbfs/pi...,NaN,Open Use,https://opendata.swiss/en/dataset/standorte-un...
788,[PickeBike],/v2/networks/pickebike-fribourg,pickebike-fribourg,PickeBike Fribourg,Fribourg,CH,46.803900,7.150500,https://github.com/SFOE/sharedmobility/blob/ma...,https://gbfs.prod.sharedmobility.ch/v2/gbfs/pi...,NaN,Open Use,https://opendata.swiss/en/dataset/standorte-un...


790 Cities in lots of countries, I want to find Toronto, Canada

In [11]:
# Country is a two letter symbol so search Canada with CA
df_networks_raw[df_networks_raw['location.country'] == 'CA']

# Toronto is found under 'Toronto, ON' not 'Toronto'

,company,href,id,name,location.city,location.country,location.latitude,location.longitude,source,gbfs_href,ebikes,license.name,license.url
70,"[Motivate International, Inc., PBSC Urban Solu...",/v2/networks/bixi-montreal,bixi-montreal,Bixi,"Montréal, QC",CA,45.508693,-73.553928,NaN,https://gbfs.velobixi.com/gbfs/gbfs.json,True,NaN,NaN
72,"[Motivate International, Inc., PBSC Urban Solu...",/v2/networks/bixi-toronto,bixi-toronto,Bike Share Toronto,"Toronto, ON",CA,43.653226,-79.383184,NaN,https://tor.publicbikesystem.net/ube/gbfs/v1/,NaN,NaN,NaN
73,[Social Bicycles Inc.],/v2/networks/sobi-hamilton,sobi-hamilton,SoBi,"Hamilton, ON",CA,43.256436,-79.869297,NaN,https://hamilton.socialbicycles.com/opendata/g...,NaN,NaN,NaN
199,"[Vanncouver Bike Share Inc., CycleHop LLC, Cit...",/v2/networks/mobibikes,mobibikes,Mobi,Vancouver,CA,49.282700,-123.120700,NaN,https://vancouver-gbfs.smoove.pro/gbfs/2/gbfs....,NaN,NaN,NaN
504,[PBSC Urban Solutions],/v2/networks/avelo-quebec,avelo-quebec,àVélo,Québec,CA,46.813050,-71.216800,NaN,https://quebec.publicbikesystem.net/customer/g...,NaN,NaN,NaN
505,[PBSC Urban Solutions],/v2/networks/acces-velo-saguenay,acces-velo-saguenay,Accès Vélo,Saguenay,CA,48.433333,-71.083333,NaN,https://saguenay.publicbikesystem.net/customer...,NaN,NaN,NaN


Parse through the response to get the details you want for the bike stations in that city (latitude, longitude, number of bikes).

In [17]:
# we have the href for toronto /v2/networks/bixi-toronto and according to api documentation we make the call by adding fields ?field=list,of,fields

href_toronto = "/v2/networks/bixi-toronto"

response = requests.request('GET', url="https://api.citybik.es/v2/networks/bixi-toronto?fields=stations")
data_stations = response.json()


**Note on CityBike API Call:**
- Current call was made at 3pm on Saturday afternoon, early August
- Warm clear weather, holiday weekend, lots of recreation going on

- Secondary call made on holiday Monday at 1:30pm - most establishments (supermarkets, some cafes and bars/restaurants) closed, less downtown activity

Put your parsed results into a DataFrame.

In [19]:
df_toronto_stations = pd.json_normalize(data_stations['network']['stations'])
df_toronto_stations

,empty_slots,free_bikes,id,latitude,longitude,name,timestamp,extra.address,extra.altitude,extra.ebikes,extra.has_ebikes,extra.last_updated,extra.normal_bikes,extra.payment,extra.payment-terminal,extra.renting,extra.returning,extra.slots,extra.uid,extra.post_code
0,6,13,fb337bbed72e2be090071e199899b2be,43.665269,-79.319796,Queen St E / Woodward Ave,2024-08-10T17:09:45.718000Z,Queen St E / Woodward Ave,0.0,1,True,1723309659,12,"[key, transitcard, creditcard, phone]",True,1,1,19,7303,NaN
1,14,1,4ff88d5880e71aa40d34cfe5d09b0ca7,43.671420,-79.445947,Primrose Ave / Davenport Rd,2024-08-10T17:09:45.729000Z,Primrose Ave / Davenport Rd,0.0,0,True,1723309646,1,"[key, transitcard, creditcard, phone]",True,1,1,15,7301,NaN
2,14,8,a09c67c0b419654d907c9134b108e328,43.666224,-79.317693,Queen St E / Rhodes Ave,2024-08-10T17:09:45.762000Z,Queen St. E / Rhodes Ave.,0.0,0,True,1723309681,8,"[key, transitcard, creditcard, phone]",True,1,1,23,7309,NaN
3,33,2,d6a9daee68070a8b106cfb598d81308c,43.653236,-79.376716,Bond St / Queen St E,2024-08-10T17:09:45.922000Z,Bond St / Queen St E,0.0,0,True,1723309700,2,"[key, transitcard, creditcard, phone]",True,1,1,37,7045,NaN
4,34,0,8f8af40d9388c8a3962559e8681d3db7,43.663722,-79.380288,Church St / Alexander St,2024-08-10T17:09:45.927000Z,Church St / Alexander St,0.0,0,True,1723309710,0,"[key, transitcard, creditcard, phone]",True,1,1,35,7044,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
823,1,8,2e10ae28a97f89714987729c5d93ebfd,43.673927,-79.460685,St. Clair Ave W / Osler St,2024-08-10T17:09:50.654000Z,St. Clair Ave W / Osler St,0.0,3,True,1723309714,5,"[key, transitcard, creditcard, phone]",True,1,1,11,7771,M6N 1J4
824,9,3,f7751b356cdb61712f43b2e083e6de4f,43.648131,-79.388581,Duncan St / Adelaide St W - SMART,2024-08-10T17:09:52.693000Z,19 Duncan St,0.0,0,True,1723309679,3,"[key, transitcard, phone]",False,1,1,12,7924,M5H 3G6
825,19,0,15189ae9afda0ad927bce260b5aa0ccc,43.692547,-79.463328,Eglinton Ave W / Gilbert Ave,2024-08-10T17:09:53.206000Z,2343 Eglinton Ave W,0.0,0,True,1723309703,0,"[key, transitcard, phone]",False,1,1,19,7743,M6E 2L6
826,11,0,c5d93c53f764429eff87446fec9c4067,43.695032,-79.453030,Eglinton Ave W / Ennerdale Rd,2024-08-10T17:09:53.255000Z,1963 Eglinton Ave W,0.0,0,True,1723309761,0,"[key, transitcard, phone]",False,1,1,14,7747,M6E 4E2


In [21]:
df_toronto_stations.shape

(828, 20)

In [29]:
# saving the df to csv so I can save the data retrieved
# csv_file_path = 'bike_stations_sat_3pm.csv'
# df_toronto_stations.to_csv(csv_file_path, index=False)

In [23]:
# extract the columns of interest into a new df to strip out the irrelevant/extra columns
df_toronto_stations = pd.read_csv('../data/bike_stations_sat_3pm.csv')

df_toronto = df_toronto_stations[['id', 'name', 'latitude', 'longitude', 'free_bikes', 'empty_slots']]

df_toronto['bike_availability'] = round((df_toronto['free_bikes'] / (df_toronto['free_bikes'] + df_toronto['empty_slots'])) * 100, 2)

df_toronto['ll'] = round(df_toronto['latitude'], 6).astype(str) + ',' + round(df_toronto['longitude'], 6).astype(str)

df_toronto

/var/folders/jj/bww8jnqd2lg73nm4k1z5kbmc0000gn/T/ipykernel_35912/1695161321.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_toronto['bike_availability'] = round((df_toronto['free_bikes'] / (df_toronto['free_bikes'] + df_toronto['empty_slots'])) * 100, 2)
/var/folders/jj/bww8jnqd2lg73nm4k1z5kbmc0000gn/T/ipykernel_35912/1695161321.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_toronto['ll'] = round(df_toronto['latitude'], 6).astype(str) + ',' + round(df_toronto['longitude'], 6).astype(str)


,id,name,latitude,longitude,free_bikes,empty_slots,bike_availability,ll
0,fb337bbed72e2be090071e199899b2be,Queen St E / Woodward Ave,43.665269,-79.319796,18,1,94.74,"43.665269,-79.319796"
1,4ff88d5880e71aa40d34cfe5d09b0ca7,Primrose Ave / Davenport Rd,43.671420,-79.445947,1,14,6.67,"43.67142,-79.445947"
2,a09c67c0b419654d907c9134b108e328,Queen St E / Rhodes Ave,43.666224,-79.317693,12,11,52.17,"43.666224,-79.317693"
3,d6a9daee68070a8b106cfb598d81308c,Bond St / Queen St E,43.653236,-79.376716,5,32,13.51,"43.653236,-79.376716"
4,8f8af40d9388c8a3962559e8681d3db7,Church St / Alexander St,43.663722,-79.380288,3,32,8.57,"43.663722,-79.380288"
...,...,...,...,...,...,...,...,...
821,9be5f078a1ed47fc11cd3cee45260f63,Kennedy Rd/Ranstone Gdns (Jack Goodlad Park),43.741906,-79.271819,1,10,9.09,"43.741906,-79.271819"
822,4ae37f3bddfb819954a15143d277dbd9,Eglinton Ave E / Brimley Rd,43.736953,-79.247984,8,11,42.11,"43.736953,-79.247984"
823,e7968ab22d9a15db0673f463144428eb,College Park South,43.659457,-79.382365,7,11,38.89,"43.659457,-79.382365"
824,62acc308c0f93ff09d28e06c73afc3ec,165 McRae Dr,43.705875,-79.368006,4,11,26.67,"43.705875,-79.368006"


# Data Audit #
Which columns are necessary? Check for Nulls above and NaNs

In [35]:
# Check for NaNs in the bike_availability column - this will disrupt our p-value calculations
print(df_toronto.isna().sum())

id                   0
name                 0
latitude             0
longitude            0
free_bikes           0
empty_slots          0
bike_availability    1
ll                   0
dtype: int64


In [41]:
# to handle the above, I will fill the one NaN in with the mean bike availability value:

df_toronto['bike_availability'].fillna(df_toronto['bike_availability'].mean(), inplace=True)

/var/folders/jj/bww8jnqd2lg73nm4k1z5kbmc0000gn/T/ipykernel_35912/3389857308.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_toronto['bike_availability'].fillna(df_toronto['bike_availability'].mean(), inplace=True)
/var/folders/jj/bww8jnqd2lg73nm4k1z5kbmc0000gn/T/ipykernel_35912/3389857308.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-cop

In [43]:
# check again for n/a

print(df_toronto.isna().sum())

id                   0
name                 0
latitude             0
longitude            0
free_bikes           0
empty_slots          0
bike_availability    0
ll                   0
dtype: int64


In [45]:
# check for duplicates - id of the bike stations, there should be the same number of unique ids to the length of the database

In [55]:
print(len(df_toronto))
# there are 826 bike stations

826


In [61]:
df_toronto['id'].is_unique

# all IDs are unique - no duplicate stations

True

In [35]:
# Saving the file below

#csv_file_path = 'bike_stations_sat_3pm_clean.csv'
#df_toronto.to_csv(csv_file_path, index=False)

**Data Obtained**
- I have saved bike_stations_sat_3pm.csv and bike_stations_sat_3pm_clean to my github repository to save the data for prime time saturday afternoon as I feel the time of day/week does have an impact on bike availability that we will see in the model. I will try this with a less prime time such as monday early afternoon (everyone is at work) or tuesday morning (typically an "in-office" day in Toronto) to see if there's a noticeable difference

- Data from bike stations at 1pm on Monday on a holiday weekend - the majority of establishments are closed in Toronto and the city is noticeably quieter than saturday (when the previous data was taken)

Data Audit:
- One station has 0 bikes / 0 free (maybe out of service?) so it was returning a NaN, which i changed to the average bike availbility.
- no duplicate stations were found (all IDs were unique)

**Thoughts on Data viz**
- The citybik.es website visualizes the occupancy of the stations on the front-end website, green with many bikes available, yellow has few bikes available and red has no bikes available. Visually there are a lot more greens in major downtown arteries than in the uptown and fringe areas.
- We should look at an sns heat map of occupancy rate and see if it tracks to major downtown areas. The visualization should be latitude vs. Longitude with the colour of each dot representing the occupancy rate of the bike stations.
- Lat/Long colour viz could be enhanced with size of the points indicating number of amenities returned by the Foursquare API call.
- If we want, we could signpost major arteries in Toronto e.g. Yonge/King, Yonge/Bloor, Bathurst/College as we expect more traffic there than other places generally in Downtown